In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from datasist.structdata import detect_outliers

%matplotlib inline
sns.set(rc={'figure.figsize': [10, 10]}, font_scale=1.3)
pd.set_option('mode.chained_assignment', None)

In [2]:
df = pd.read_csv("Sample - Superstore.csv", encoding='windows-1252')
df.head()

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,...,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,1,CA-2016-152156,11/08/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600,2,0.00,41.9136
1,2,CA-2016-152156,11/08/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400,3,0.00,219.5820
2,3,CA-2016-138688,06/12/2016,6/16/2016,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,...,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200,2,0.00,6.8714
3,4,US-2015-108966,10/11/2015,10/18/2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.0310
4,5,US-2015-108966,10/11/2015,10/18/2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680,2,0.20,2.5164


In [3]:
inputs= ['Quantity', 'Discount', 'Profit', 'Order Date','Ship Date', 'Ship Mode', 'Segment', 'State' , 'Region', 'Category', 
         'Sub-Category', ]

In [4]:
def process_ship_Mode (ShipMode):
    if ShipMode == 'Second Class' :
        return [1,0,0,0]
    elif ShipMode == 'Standard Class' :
        return [0,1,0,0]
    elif ShipMode == 'First Class' :
        return [0,0,1,0]
    elif ShipMode == 'Same Day' :
        return [0,0,0,1]
    

def process_Segment (Segment):
    if Segment == 'Consumer' :
        return [1,0,0]
    elif Segment == 'Corporate' :
        return [0,1,0]
    elif Segment == 'Home Office' :
        return [0,0,1]
    
    
def process_State (State):
    if State  == 'Kentucky' :
        return [1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
    elif State == 'California' :
        return [0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
    elif State == 'Florida' :
        return [0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
    elif State == 'North Carolina' :
        return [0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
    elif State == 'Washington' :
        return [0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
    elif State == 'Texas' :
        return [0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
    elif State == 'Wisconsin' :
        return [0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
    elif State == 'Utah' :
        return [0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
    elif State == 'Nebraska' :
        return [0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
    elif State == 'Pennsylvania' :
        return [0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
    elif State == 'Illinois' :
        return [0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
    elif State == 'Minnesota' :
        return [0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
    elif State == 'Michigan' :
        return [0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
    elif State == 'Delaware' :
        return [0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
    elif State == 'Indiana' :
        return [0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
    elif State == 'New York' :
        return [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
    elif State == 'Arizona' :
        return [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
    elif State == 'Virginia' :
        return [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
    elif State == 'Tennessee' :
        return [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
    elif State == 'Alabama' :
        return [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
    elif State == 'South Carolina' :
        return [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
    elif State == 'Oregon' :
        return [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
    elif State == 'Colorado' :
        return [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
    elif State == 'Iowa' :
        return [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
    elif State == 'Ohio' :
        return [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
    elif State == 'Missouri' :
        return [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
    elif State == 'Oklahoma' :
        return [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
    elif State == 'New Mexico' :
        return [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
    elif State == 'Louisiana' :
        return [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
    elif State == 'Connecticut' :
        return [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
    elif State == 'New Jersey' :
        return [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
    elif State == 'Massachusetts' :
        return [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
    elif State == 'Georgia' :
        return [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
    elif State == 'Nevada':
        return [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
    elif State == 'Rhode Island':
        return [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
    elif State == 'Mississippi':
        return [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0]
    elif State == 'Arkansas':
        return [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0]
    elif State == 'Montana':
        return [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0]
    elif State == 'New Hampshire':
        return [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0]
    elif State == 'Maryland':
        return [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0]
    elif State == 'District of Columbia':
        return [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0]
    elif State == 'Kansas':
        return [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0]
    elif State == 'Vermont':
        return [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0]
    elif State == 'Maine':
        return [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0]
    elif State == 'South Dakota':
        return [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0]
    elif State == 'Idaho':
        return [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0]
    elif State == 'North Dakota':
        return [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0]
    elif State == 'Wyoming':
        return [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0]
    elif State == 'West Virginia':
        return [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1]

    
def process_Region(Region):
    if Region == 'South':
        return [1,0,0,0]
    elif Region == 'West' :
        return [0,1,0,0]
    elif Region == 'Central' :
        return [0,0,1,0]
    elif Region == 'East' :
        return [0,0,0,1]
        
        
def process_Category(Category):
    if Category == 'Furniture' :
        return [1,0,0]
    elif Category == 'Office Supplies' :
        return [0,1,0]
    elif Category == 'Technology' :
        return [0,0,1]

def process_Sub_Category(SubCategory):
    if SubCategory == 'Bookcases' :
        return [1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
    elif SubCategory == 'Chairs' :
        return [0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
    elif SubCategory == 'Labels' :
        return [0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
    elif SubCategory == 'Tables' :
        return [0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0]
    elif SubCategory == 'Storage' :
        return [0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0]
    elif SubCategory == 'Furnishings' :
        return [0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0]
    elif SubCategory == 'Art' :
        return [0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0]
    elif SubCategory == 'Phones' :
        return [0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0]
    elif SubCategory == 'Binders' :
        return [0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0]
    elif SubCategory == 'Appliances' :
        return [0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0]
    elif SubCategory == 'Paper' :
        return [0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0]
    elif SubCategory == 'Accessories' :
        return [0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0]
    elif SubCategory == 'Envelopes':
        return [0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0]
    elif SubCategory == 'Fasteners':
        return [0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0]
    elif SubCategory == 'Supplies':
        return [0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0]
    elif SubCategory == 'Machines':
        return [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0]
    elif SubCategory == 'Copiers':
        return [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1]
    

In [5]:
def preprocess_data(data):
    
    quntity      = data['Quantity']
    discount     = data['Discount']
    profit       = data['Profit']
    Order_Date_year = data['Order Date_year']
    Order_Date_month = data['Order Date_month']
    Order_Date_day = data['Order Date_day']
    Ship_Date_year = data['Ship Date_year']
    Ship_Date_month = data['Ship Date_month']
    Ship_Date_day  = data['Ship Date_day']
    ship_mode    = process_ship_Mode(data['ShipMode'])
    segment      = process_Segment(data['Segment'])
    state        = process_State(data['State'])
    region       = process_Region(data['Region'])
    category     = process_Category(data['Category'])
    sub_category = process_Sub_Category(data['SubCategory'])
    final_data   = [quntity,discount,profit,Order_Date_year,Order_Date_month,Order_Date_day,Ship_Date_year,Ship_Date_month,Ship_Date_day]+ ship_mode + segment + state + region + category + sub_category
    
    return final_data

In [6]:
data = {'Quantity':25 ,'Discount': 45.98, 'Profit':345.67,'Order Date_year': 2015 ,
        'Order Date_month':5 , 'Order Date_day':8 , 'Ship Date_year': 2015, 
        'Ship Date_month':7,'Ship Date_day':22,'ShipMode':'Second Class','Segment':'Consumer',
        'State':'Kansas','Region':'East' ,'Category':'Office Supplies','SubCategory':'Supplies' }

In [7]:
final_data = preprocess_data(data)
final_data

[25,
 45.98,
 345.67,
 2015,
 5,
 8,
 2015,
 7,
 22,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0]

In [8]:
import joblib

In [9]:
model = joblib.load('model.h5')
scaler = joblib.load('scaler.h5')

In [10]:
final_data = scaler.transform([final_data])
final_data

array([[ 9.54759119e+00,  2.21717834e+02,  1.34479104e+00,
        -6.37505291e-01, -8.57720148e-01, -8.60355357e-01,
        -6.49316811e-01, -2.24753005e-01,  6.87947133e-01,
         2.33127615e+00, -2.40381598e-01, -4.86384722e-01,
        -1.22132706e+00,  9.61421693e-01, -6.59391611e-01,
        -4.64233031e-01, -8.43448998e-02, -1.54888204e-01,
        -7.53867603e-02, -4.97460137e-01, -1.34532543e-01,
        -9.18112233e-02, -9.99405293e-02, -3.43320448e-02,
        -1.99779506e-01, -1.37673290e-01, -4.34423475e-02,
        -2.26069760e-01, -1.21716124e-01, -5.64699109e-02,
        -4.97841440e-02, -1.18692334e-01, -6.52405194e-02,
        -3.07038955e-02, -9.81221521e-02, -1.19201322e-01,
        -1.61917351e-01, -9.50172234e-02, -7.29799568e-02,
        -8.07301671e-02, -3.76133044e-02, -6.52405194e-02,
        -6.05227533e-02, -5.32309131e-02, -1.14014928e-01,
        -6.14948121e-02, -3.58846435e-01, -1.58047090e-01,
        -2.65872206e-02, -2.24338330e-01, -8.14655986e-0

In [11]:
model.predict(final_data)[0]

2112.7046